# Homework #5

Carina Cheng

# 1) 

In [1]:
import pandas as pd
import sqlite3
import difflib

# Find top 50 airports from csv file
top_airports = '/Users/carinacheng/Desktop/Carina/UCBClasses/python-seminar/Homeworks/hw_5/hw_5_data/top_airports.csv'
df = pd.read_csv(top_airports)
df.sort_values(by='Enplanements',ascending=False) # sort by number of airplane passengers
df.head(50) # top 50
cities = df['City'].tolist() # list of cities
airports = df['Airport'].tolist() # list of airports
codes = df['ICAO'].tolist() # list of airport codes

# Create empty sqlite3 table for airport information
connection = sqlite3.connect("airports.db") # new database
cursor = connection.cursor() 
cursor.execute("""DROP TABLE IF EXISTS airports""") # drop table if exists
sql_cmd = """CREATE TABLE airports (id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT, city TEXT, latitude FLOAT, longitude FLOAT)""" # create table named 'airports'
cursor.execute(sql_cmd) 

# Read latitude/longitude data
info_airports = '/Users/carinacheng/Desktop/Carina/UCBClasses/python-seminar/Homeworks/hw_5/hw_5_data/ICAO_airports.csv'
df = pd.read_csv(info_airports, index_col='name') # index col is airport name
names = df.index.tolist()

# Populate table
print('Top 50 Airports: \n')
for aa,airport in enumerate(airports): # loop through top 50 airports
    try: # if airport names match exactly in both files 
        lat = df.loc[airport]['latitude_deg'] # get latitude
        lon = df.loc[airport]['longitude_deg'] # get longitude
    except: # find closest matching airport name in second file
        airport = airport.split('(')[0] # remove parentheticals in name
        name = difflib.get_close_matches(airport,names)[0] # take closest matching airport name in second file
        lat = df.loc[name]['latitude_deg'] # get latitude
        lon = df.loc[name]['longitude_deg'] # get longitude
    print(str(aa+1)+'/'+str(len(airports))+': '+airport+' ('+cities[aa]+')')
    data = (airport, cities[aa], lat, lon) # data to save
    sql_cmd = ("INSERT INTO airports (name, city, latitude, longitude) VALUES " + str(data))
    cursor.execute(sql_cmd) # put data into table

# Test query
#sql_cmd = "SELECT * FROM airports WHERE latitude > 40" # query
#cursor.execute(sql_cmd)
#db_info = cursor.fetchall() # get all entries from the previous execution
#for entry in db_info: 
#    print(entry)

# Save and close
connection.commit()
connection.close()

Top 50 Airports: 

1/50: Hartsfield-Jackson Atlanta International Airport (Atlanta)
2/50: Chicago O'Hare International Airport (Chicago)
3/50: Los Angeles International Airport (Los Angeles)
4/50: Dallas/Fort Worth International Airport (Dallas-Fort Worth)
5/50: Denver International Airport (Denver)
6/50: John F. Kennedy International Airport (New York)
7/50: San Francisco International Airport (San Francisco)
8/50: George Bush Intercontinental Airport (Houston)
9/50: McCarran International Airport (Las Vegas)
10/50: Phoenix Sky Harbor International Airport (Phoenix)
11/50: Charlotte/Douglas International Airport (Charlotte)
12/50: Miami International Airport (Miami)
13/50: Orlando International Airport (Orlando)
14/50: Newark Liberty International Airport (Newark)
15/50: Detroit Metropolitan Wayne County Airport (Detroit / Romulus)
16/50: Minneapolis-St. Paul International Airport  (Minneapolis)
17/50: Seattle-Tacoma International Airport (Seattle / Tacoma (SeaTac))
18/50: Philadelphi

# 2) and 3)

In [2]:
from bs4 import BeautifulSoup
import requests

# Create empty sqlite3 table for weather information
connection = sqlite3.connect("weather.db") # new database
cursor = connection.cursor() 
cursor.execute("""DROP TABLE IF EXISTS weather""") # drop table if exists
sql_cmd = """CREATE TABLE weather (id INTEGER PRIMARY KEY AUTOINCREMENT,
    airport TEXT, date TEXT, min_temp FLOAT, max_temp FLOAT, humidity FLOAT, precipitation FLOAT)""" # create table named 'weather'
cursor.execute(sql_cmd) 

# Pull historical weather data 
# Loop over airports and dates in order to populate tables
date_i = 86 # these are the initial indices for pulling data off the wunderground table
maxt_i = 87
mint_i = 89
hum_i = 94
prec_i = 105
for aa,airport in enumerate(airports[:3]): # loop over airports
    print(aa+1,'/',len(airports))
    code = codes[aa] # airport code
    # get wunderground history for 10 years
    weather_site = 'https://www.wunderground.com/history/airport/'+code+'/2017/4/13/CustomHistory.html?dayend=5&monthend=3&yearend=2018'
    req = requests.get(weather_site)
    soup = BeautifulSoup(req.text,"html.parser")
    page = soup.findAll('td')
    for row in range(int((len(page)-date_i)/21)): # loop over table rows (total number of entries divided by entires per row)
        try: # find date, temperatures, humidity, and precipitation
            url = page[date_i].findAll('a')[0]['href'].split('/Daily')[0] # parse url
            dat = "/".join(url.split('/')[-3:]) # parse date from url
            max_temp = float(page[maxt_i].get_text()) # get max temp
            min_temp = float(page[mint_i].get_text()) # get min temp
            precipitation = page[prec_i].get_text() # get precipitation
            humidity = float(page[hum_i].get_text()) # get humidity
            if 'T' in precipitation: precipitation = 0.0 # T for 'trace' precipitation ~ 0.0 in. of rainfall
            else: precipitation = float(precipitation) # else it's a float
            data = (str(airport), dat, min_temp, max_temp, humidity, precipitation) # data to save
            sql_cmd = ("INSERT INTO weather (airport, date, min_temp, max_temp, humidity, precipitation) VALUES " + str(data))
            cursor.execute(sql_cmd) # put data into table
            connection.commit() # save
        # increment indices
        except: pass # if a row does not match that format (a 'heading' row, for example)
        date_i += 21 # 21 is the number of entries in each row
        maxt_i += 21
        mint_i += 21
        hum_i += 21
        prec_i += 21
connection.close()

1 / 50
2 / 50
3 / 50


# 4)

In [3]:
def get_lat(a1,a2):
    """
    Extracts the latitude entries for two airports.
        Inputs: a1 (airport 1), a2 (airport 2)
        Outputs: lat1 (latitude 1), lat2 (latitude 2)
    """
    try: # try single quotes around airport name
        lat1 = cursor2.execute("SELECT latitude FROM airports WHERE name == '" + a1 + "'").fetchall()[0][0]
    except: # if that doesn't work, try double quotes
        lat1 = cursor2.execute('SELECT latitude FROM airports WHERE name == "' + a1 + '"').fetchall()[0][0]
    try: # try single quotes around airport name
        lat2 = cursor2.execute("SELECT latitude FROM airports WHERE name == '" + a2 + "'").fetchall()[0][0]
    except: # if that doesn't work, try double quotes
        lat2 = cursor2.execute('SELECT latitude FROM airports WHERE name == "' + a2 + '"').fetchall()[0][0]
    return lat1,lat2

def get_data(a1,a2,key='precipitation'):
    """
    Extracts entries for two airports.
        Inputs: a1 (airport 1), a2 (airport 2), key (database key)
        Outputs: data1, data2
    """
    try: # try single quotes around airport name
        sql_cmd = "SELECT " + key + " FROM weather WHERE airport == '" + a1 + "'" # query
        cursor.execute(sql_cmd)
        dat1 = cursor.fetchall()
    except: # if that doesn't work, try double quotes
        sql_cmd = 'SELECT ' + key + ' FROM weather WHERE airport == "' + a1 + '"'
        cursor.execute(sql_cmd)
        dat1 = cursor.fetchall()
    try: # try single quotes around airport name
        sql_cmd = "SELECT " + key + " FROM weather WHERE airport == '" + a2 + "'" # query
        cursor.execute(sql_cmd)
        dat2 = cursor.fetchall()
    except: # if that doesn't work, try double quotes
        sql_cmd = 'SELECT ' + key + ' FROM weather WHERE airport == "' + a2 + '"'
        cursor.execute(sql_cmd)
        dat2 = cursor.fetchall()
    data1, data2 = [], []
    for d in dat1: data1.append(d[0])
    for d in dat2: data2.append(d[0])
    return data1,data2

def compute_corr(a1,a2,offset=0):
    """
    Computes correlation of precipitation and max temperature between two airports.
        Inputs: a1 (airport1), a2 (airport2), offset (daily offset to compute correlation for)
        Outputs: corr_prec (correlation of precipitation), corr_temp (correlation of max temperature)
    """
    lat1,lat2 = get_lat(a1,a2) # latitudes
    maxt1,maxt2 = get_data(a1,a2,key='max_temp') # max temps
    prec1,prec2 = get_data(a1,a2,key='precipitation') # precipitations
    # figure out which dataset needs the offset
    # apply offset
    # compute correlation
    
# Connect to databases
connection = sqlite3.connect("weather.db") # load database   
cursor = connection.cursor() 
connection2 = sqlite3.connect("airports.db") # load database   
cursor2 = connection2.cursor() 

# Loop through all airport pairs
stop = 3 # remove this later
for aa,a1 in enumerate(airports[:stop]): # loop through airports
    for a2 in airports[aa+1:stop]: # loop through all other airports to find pairs
        corr = compute_corr(a1,a2,offset=1) # 1-day offset

# 5)